# Google Colabを利用してますでしょうか？
以下のコードを実行してください。
- UPSTAGE_API_KEYを取得・書き換え

```sh
# 🔑 APIキー設定
# # https://console.upstage.ai/api-keys
import os
os.environ["UPSTAGE_API_KEY"] = "YOUR_API_KEY"
print("🔐 Upstage APIキーを設定しました")
```

他の設定は入りません。

In [ ]:
import os

# ===============================
# 🧪 weaveのインストール
# ===============================
print("📦 weave をインストール中...")
!pip install -q weave==0.52.9

# ===============================
# 🔑 Upstage APIキーの設定
# ===============================
# # https://console.upstage.ai/api-keys
os.environ["UPSTAGE_API_KEY"] = "YOUR_API_KEY"
print("🔐 Upstage APIキーを設定しました")

# ===============================
# 📁 パス設定
# ===============================
ZIP_URL = "https://github.com/nhandsome-new/fc_2025_upstage_handson/raw/main/prep_google_colab/demo_imgs.zip"
ZIP_FILE = "demo_imgs.zip"
EXTRACT_DIR = "/content/demo_imgs"

# ===============================
# 📂 フォルダがあるかチェック
# ===============================
if not os.path.exists(EXTRACT_DIR):
    print("📂 フォルダが存在しません。初回セットアップを開始します...")

    # 🧭 1) 既存のフォルダ・ZIPファイルを削除（存在しなくてもOK）
    print("🧭 既存のフォルダ・ZIPファイルを削除中...")
    !rm -rf {EXTRACT_DIR} || true
    !rm -f {ZIP_FILE} || true

    # ⬇️ 2) wgetでZIPファイルをダウンロード
    print(f"⬇️ ZIPファイルをダウンロード中: {ZIP_URL}")
    !wget -q -O {ZIP_FILE} {ZIP_URL}
    print("✅ ダウンロード完了")

    # 📦 3) ZIPファイルを解凍
    print("📦 ZIPファイルを解凍中...")
    !unzip -q {ZIP_FILE} -d .
    print("✅ 解凍完了")

    # 🧼 4) ZIPファイルを削除
    print("🧼 ZIPファイルを削除中...")
    !rm {ZIP_FILE}
    print("✅ ZIPファイル削除完了")

else:
    print("✅ 既にフォルダが存在するため、ダウンロードと解凍はスキップします。")

# ===============================
# 📂 展開結果の確認
# ===============================
print(f"📂 展開結果の確認: {EXTRACT_DIR}")
!ls -l {EXTRACT_DIR}


In [3]:
# ======================================================
# 📦 1. 必要なライブラリのインポート
# ======================================================
import weave
from weave import Content
import os
import re
import json
import requests
from datetime import datetime
from IPython.display import display, Image, HTML
from typing import Annotated

# ======================================================
# 🚀 2. Weaveの初期化
# ======================================================
weave.init("UPSTAGE-HANDSON-STEP2")

# ======================================================
# 🔐 3. APIキーの設定
# https://console.upstage.ai/api-keys
# ======================================================
# os.environ["UPSTAGE_API_KEY"] = 'YOUR_API_KEY' # ← 実際のキーをここに入れる

# ======================================================
# ✅ 4. 初期化確認
# ======================================================
print("📚 ライブラリ準備完了!")
print("🎯 Weaveが初期化されました - すべての処理が自動追跡されます")
print(f"🔑 UPSTAGE_API_KEY: {'設定済み' if 'UPSTAGE_API_KEY' in os.environ and os.environ['UPSTAGE_API_KEY'] != 'YOUR_API_KEY' else '❌未設定'}")

# ======================================================
# 🧾 5. Document Parse 関数
# ======================================================
@weave.op(name=f"document_parser_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
def upstage_document_parse(file_path: Annotated[str, Content]) -> str:
    """
    📄 文書をHTMLに変換する関数
    - Upstage Document AI API を利用
    - ファイルを送信して OCR + HTML 化
    """
    # Step 1. APIキーを環境変数から取得
    api_key = os.environ.get("UPSTAGE_API_KEY")
    if not api_key:
        raise ValueError("❌ UPSTAGE_API_KEY が環境変数に設定されていません。")

    # Step 2. 画像ファイルをバイナリで読み込み
    with open(file_path, 'rb') as file:
        files = {
            "document": (os.path.basename(file_path), file, "image/jpeg")
        }
        data = {
            "model": "document-parse-nightly",
            "ocr": "auto",
            "output_formats": "html"
        }

        # Step 3. APIリクエスト送信
        response = requests.post(
            "https://api.upstage.ai/v1/document-ai/document-parse",
            headers={"Authorization": f"Bearer {api_key}"},
            files=files,
            data=data
        )

    # Step 4. 結果をJSONで取得してHTML部分を返す
    result = response.json()
    return result.get("content", {}).get("html", "")


# ======================================================
# ☀️ 6. LLM 応答生成関数
# ======================================================
@weave.op(name=f"llm_generator_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
def upstage_solar_pro2(messages: list) -> str:
    """
    🧠 LLMで応答を生成する関数
    - Solar Pro2 モデルを使用
    - messages には system / user ロールのプロンプトを含む
    """
    # Step 1. APIキーを環境変数から取得
    api_key = os.environ.get("UPSTAGE_API_KEY")
    if not api_key:
        raise ValueError("❌ UPSTAGE_API_KEY が環境変数に設定されていません。")

    # Step 2. リクエストペイロードの構築
    payload = {
        "model": "solar-pro2",
        "messages": messages,
        "temperature": 0.1,
        "max_tokens": 10000
    }

    # Step 3. APIリクエスト送信
    response = requests.post(
        "https://api.upstage.ai/v1/solar/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=payload
    )

    # Step 4. 応答メッセージ部分のみを抽出して返す
    result = response.json()
    return result["choices"][0]["message"]["content"]


# ======================================================
# 🖼 7. ファイル情報の表示関数
# ======================================================
def display_document_info(file_path: str, title: str = ""):
    """
    📌 文書ファイルの基本情報を表示
    - ファイル名とサイズを出力
    - JPG/PNG形式の場合は画像を表示
    """
    if os.path.exists(file_path):
        # ファイル情報の出力
        print(f"\n📄 {title}")
        print(f"   ファイル名: {os.path.basename(file_path)}")
        print(f"   サイズ: {os.path.getsize(file_path):,} バイト")

        # 画像プレビュー（対応形式のみ）
        file_ext = os.path.splitext(file_path)[1].lower()
        if file_ext in ['.jpg', '.jpeg', '.png']:
            display(Image(file_path, width=300))
    else:
        print(f"❌ ファイルが見つかりません: {file_path}")

# ======================================================
# 🪜 8. JSON抽出関数
# ------------------------------------------------------
# - LLMの応答テキストから JSON フォーマット部分のみを抽出
# ======================================================
@weave.op(name=f"json_extractor_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
def extract_json_from_llm_response(content: str):
    content = content.strip()
    match = re.search(r"```json\s*(\{[\s\S]*?\})\s*```", content)
    if match:
        content = match.group(1)
    return json.loads(content)


weave: Logged in as Weights & Biases user: nhandsome.
weave: View Weave data at https://wandb.ai/nhandsome/UPSTAGE-HANDSON-STEP2/weave


📚 ライブラリ準備完了!
🎯 Weaveが初期化されました - すべての処理が自動追跡されます
🔑 UPSTAGE_API_KEY: 設定済み


# 🧠 Upstage Document Parse / LLM 実活用の考え方  
## シナリオ：保険会社 — 事故状況報告書からの情報抽出

### 📝 背景
- 保険会社では、事故報告書のフォーマットは **会社ごとに異なる**。
- 担当者が業務で必要とする情報は以下の3つ：
  - 受傷者の情報
  - 受傷（事故）の情報
  - 免許証の情報
- テンプレートの違いを吸収しつつ、必要な情報を自動で抽出する仕組みが重要。

---

![](./docs_imgs/Step_2.png)

---

## 🧪 ステップ①：単純な情報抽出
最初のステップでは、LLMに対してシンプルに以下のように指示します：

```yaml
PROMPT = """
ドキュメント内容を見て、
  - 受傷者の情報
  - 受傷（事故）の情報
  - 免許証の情報
を抽出してください。
"""
```

In [4]:
# ======================================================
# 📁 1. 学習・テスト用のサンプル文書パスを設定
# ======================================================
SAMPLE_DOCUMENTS = [
    "./demo_imgs/information_extraction/_Himawari_Accident_Report.jpg",
]
sample_file = SAMPLE_DOCUMENTS[0]

# ======================================================
# 🖼 2. 対象文書の基本情報とプレビューを表示
# ======================================================
# display_document_info(sample_file, "テストサンプル文書")

# ======================================================
# 📄 3. Document Parse 実行
# ======================================================
print("\n📋 Document Parse:")
html_result = upstage_document_parse(sample_file)

# ======================================================
# 📝 4. LLMに渡すプロンプトを定義
# ------------------------------------------------------
# - 単純な情報抽出プロンプト
# ======================================================
PROMPT = """
ドキュメント内容を見て、
  - 受傷者の情報
  - 受傷（事故）の情報
  - 免許証の情報
を抽出してください。
"""

# ======================================================
# 🧠 5. LLMに送信するメッセージの準備
# ======================================================
explanation_messages = [
    {
        "role": "system",
        "content": "あなたは、HTML形式のドキュメント構造を理解し、ユーザーの指示に従ってタスクを実行するアシスタントです。必ず日本語で答えてください。"
    },
    {
        "role": "user",
        "content": f"# ドキュメント内容(HTML):\n{html_result}\n\n# プロンプト指示:\n{PROMPT}"
    }
]

# ======================================================
# ☀️ 6. Solar Pro2 による LLM 推論の実行
# ======================================================
print("\n🧠 LLMの答え:")
result = upstage_solar_pro2(explanation_messages)
print(result)



📋 Document Parse:


weave: 🍩 https://wandb.ai/nhandsome/UPSTAGE-HANDSON-STEP2/r/call/0199e7e4-add5-7287-a27e-8f17bd9f6d5f



🧠 LLMの答え:


weave: 🍩 https://wandb.ai/nhandsome/UPSTAGE-HANDSON-STEP2/r/call/0199e7e4-c883-7140-bbcf-7c943cac15fd


### 抽出結果

#### 1. 受傷者の情報
- **氏名**: サンプル 太郎  
- **性別**: 男  
- **生年月日**: 昭和 45年 4月 3日  
- **受取人 (請求権者)**: サンプル 太郎  
- **親権者/後見人**: 記載なし  

#### 2. 受傷（事故）の情報
- **受傷日時**: 令和元年 9月 15日 午後 3時 00分頃  
- **受傷場所**: 東京都中央区京橋3-30-90  
- **受傷原因/状況**:  
  - 銀座方面に向かって普通乗用車を運転中に、左脇道より一旦停止せずに出てきた相手方の小型トラックに衝突され、頭部を強打。  
- **事故状況**:  
  - **当事者役割**: 運転中  
  - **飲酒有無**: 無  
  - **警察届出**: 有 (中央警察署へ提出)  

#### 3. 免許証の情報
- **免許証番号**: 10101112234  
- **免許証種類**: 普通  
- **有効期限**: 令和2年 3月 3日  

### 備考
- ドキュメントは「事故状況報告書」であり、保険会社「ひまわり生命保険株式会社」に提出されたものです。  
- 保険金関連番号: 19. 10 (891435) (フッター記載)。


weave: 🍩 https://wandb.ai/nhandsome/UPSTAGE-HANDSON-STEP2/r/call/0199e7e4-ec64-77f4-8f1b-7906428d2ec0
weave: 🍩 https://wandb.ai/nhandsome/UPSTAGE-HANDSON-STEP2/r/call/0199e7e5-00fc-7c1e-8c95-5ad982b41dcf


## 🏗 実活用・開発で考慮すべきポイント

## 🧭 シナリオ：精度・表記方法・システム連携を担保する抽出設計
個人で試すレベルではシンプルなプロンプトでもある程度機能します。  
しかし、実際の業務現場では以下のような要件を満たす必要があります：

- 抽出結果の **精度** を安定させたい  
- **表記方法** を統一して社内システムで扱いやすくしたい  
- 他システムとの **連携可能な形式**（構造化データ）で出力したい  

👉 これらを制御することで、業務フローに組み込める実用的な情報抽出パイプラインを構築できます。

---

### 1. 情報抽出プロセスの段階的アプローチ
- LLMに対していきなり抽出を指示するのではなく、  
  **まずHTML構造を整理 → その上で情報抽出** という2段階に分けると精度が安定します。
- 例：
  - Step 1: HTMLを「必要な情報を特定しやすい構造」に整理
  - Step 2: 抽出対象を明確に指示

---

### 2. 必要な情報の明確な定義
- 担当者が業務で実際に必要とする情報を **事前に定義しておく** ことが重要です。
- 例：
  - 「住所」は都道府県・市区町村までで十分
  - 「日時」は `YYYY/MM/DD HH:mm` 形式
  - 「受傷の内容」と「受傷の原因」は分けて抽出
- 参考情報をプロンプト内に明示することで LLM の迷いを減らすことができます。

---

### 3. 出力形式の定義
- 抽出した情報は、**明確な形式** で返すようにします。
- 例：JSON形式での出力指定
  ```json
  {
    "受傷者名前": "山田 太郎",
    "受傷者性別": "男",
    "受傷者生年月日": "1990/05/12",
    "受傷者住所": "福岡県福岡市中央区",
    "受傷の日時/日付": "2025/10/10",
    "受傷の日時/時間": "14:35",
    "受傷の場所": "福岡県福岡市博多区博多駅前3-4-5",
    "受傷の原因": "通勤中に段差で転倒",
    "受傷の内容": "右ひじ擦過傷",
    "飲酒の有無": "無",
    "警察の届出/有無": "有",
    "警察の届出/警察署名": "福岡警察署",
    "免許証番号": "123456789012",
    "免許証種類": "原付",
    "免許有効期間": "2027/05/12"
  }
  ```

In [5]:
# ======================================================
# 🪜 Step ① 情報抽出プロセスの段階的アプローチ
# ------------------------------------------------------
# - まずHTML内容を整形して、LLMが情報を取り出しやすい構造に変換
# - 抽出対象の情報は「受傷者の情報」「受傷（事故）の情報」「免許証の情報」
# ======================================================
PROMPT_1 = """
内容を変えずに、HTML内容を綺麗に整理して、新たなHTMLコードを作成してください。
- 必要な内容
  - 受傷者の情報
  - 受傷（事故）の情報
  - 免許証の情報
"""

# ======================================================
# 🪜 Step ② 必要な情報の明確な定義
# ------------------------------------------------------
# - 実務で必要な情報を明示的に指示
# - 「記載無し」などの記載ルールや日付・時間フォーマットも指定
# - 不明確な推測を防ぐことで安定した抽出結果を得る
# ======================================================
PROMPT_2 = """
情報が明確に記載されていない場合は、「記載無し」 と記載してください。
不明確な推測は行わず、記載内容のみに基づいて抽出してください。

-「受傷者住所」は 番地を含めない
-「受傷の場所」は 番地を含める
- 日付は 西暦（YYYY/MM/DD） で表記する
-「受傷の原因」「受傷の内容」は抽出情報から分けて整理する
- 時刻は 24時間表記（HH:mm）
"""

# ======================================================
# 🪜 Step ③ 出力形式の定義
# ------------------------------------------------------
# - LLMに対して出力フォーマット（JSON形式）を明示
# - 事前に構造を定義しておくことで、システムとの連携が容易になる
# ======================================================
PROMPT_3 = """
ドキュメント内容を見て、以下のフォーマットに従って JSON形式で標準化された情報を出力する担当者です。
# 出力フォーマット/例（JSON形式）

{
  "受傷者名前": "山田 太郎",
  "受傷者性別": "男",
  "受傷者生年月日": "1990/05/12",
  "受傷者住所": "福岡県福岡市中央区",
  "受傷の日時/日付": "2025/10/10",
  "受傷の日時/時間": "14:35",
  "受傷の場所": "福岡県福岡市博多区博多駅前3-4-5",
  "受傷の原因": "通勤中に段差で転倒",
  "受傷の内容": "右ひじ擦過傷",
  "飲酒の有無": "無",
  "警察の届出/有無": "有",
  "警察の届出/警察署名": "福岡警察署",
  "免許証番号": "123456789012",
  "免許証種類": "原付",
  "免許有効期間": "2027/05/12"
}
"""

# ======================================================
# 🪜 Step ④ LLMへのメッセージ構築
# ------------------------------------------------------
# - 各プロンプトを段階的に適用
#   ① HTML整形 → ② 情報定義に基づく抽出 → ③ 出力形式の指定
# ======================================================
print("\n🧠 LLMの答え:")
explanation_messages = [
    {
        "role": "system",
        "content": "あなたは、HTML形式のドキュメント構造を理解し、ユーザーのプロンプトに従ってタスクを実行するアシスタントです。必ず日本語で答えてください。"
    },
    {
        "role": "user",
        "content": f"# ドキュメント内容(HTML):\n{html_result}\n\n# プロンプト指示:\n{PROMPT_1}"
    },
    {
        "role": "user",
        "content": f"前回結果をベースに次のタスクを行ってください。\n# プロンプト指示:\n{PROMPT_2}"
    },
    {
        "role": "user",
        "content": f"前回結果をベースに次のタスクを行ってください。\n# プロンプト指示:\n{PROMPT_3}"
    }
]

# ======================================================
# 🪜 Step ⑤ Solar Pro2 での抽出実行
# ======================================================
result = upstage_solar_pro2(explanation_messages)

# ======================================================
# 🪜 Step ⑥ JSON形式への変換
# ======================================================
result_json = extract_json_from_llm_response(result)

# ======================================================
# 🪜 Step ⑦ 抽出結果の確認
# ======================================================
print(json.dumps(result_json, indent=2, ensure_ascii=False))



🧠 LLMの答え:
{
  "受傷者名前": "サンプル 太郎",
  "受傷者性別": "男",
  "受傷者生年月日": "1970/04/03",
  "受傷者住所": "記載無し",
  "受傷の日時/日付": "2019/09/15",
  "受傷の日時/時間": "15:00",
  "受傷の場所": "東京都中央区京橋3 - 30 - 90",
  "受傷の原因": "運転中に左脇道より一旦停止せずに出てきた相手方の小型トラックに衝突された",
  "受傷の内容": "頭部を強打した",
  "飲酒の有無": "無",
  "警察の届出/有無": "有",
  "警察の届出/警察署名": "中央警察署",
  "免許証番号": "10101112234",
  "免許証種類": "普通",
  "免許有効期間": "2020/03/03"
}


# 結果
## weave
- 簡単に可視化・確認可能
- text / markdown / code MODE
- jsonのテーブル化
## 実際の結果
![](./docs_imgs/Step_2_Result.png)